不管数据如何流动，yield 都是一种流程控制工具，使用它可以实现协作式多任务：协程可以把控制器让步给中心调度程序，从而激活其他的协程。

协程是指一个过程，这个过程与调用方协作，产出由调用方提供的值。

In [1]:
# 协程使用生成器函数定义
def simple_coroutine():
    print('-> coroutine started')
    x = yield  # 如果协程只需从客户那里接收数据，那么产出的值是 None（隐式指定）
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x00000215837A5DD0>

In [3]:
# 启动生成器
next(my_coro)

-> coroutine started


In [4]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

协程可以处于四个状态中的一个，当前状态可以使用 inspect.getgeneratorstate(...) 函数确定
- GEN_CREATED
    - 等待开始执行
- GEN_RUNNING
    - 解释器正在执行（只有在多线程应用中才能看到这个状态）
- GEN_SUSPENDED
    - 在 yield 表达式处暂停
- GEN_CLOSED
    - 执行结束
    
仅当协程处于暂停状态时才能调用 send 方法；始终要调用 next(my_coro) 激活协程，也可以调用 mycoro.send(None)

预激（prime）协程：让协程向前执行到第一个 yield 表达式，准备好作为活跃的协程使用

In [5]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

协程在 yield 关键字所在的位置暂停执行。在赋值语句中，= 右边的代码再赋值之前执行。

各个阶段都在 yield 表达式中结束，而且下一个阶段都从那一行代码开始，然后再把 yield 表达式的值赋值给变量。

![](https://github.com/fluentpython/images/blob/master/simple_coro2.png?raw=true)

In [6]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield  a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [7]:
my_coro2 = simple_coro2(14)

In [8]:
from inspect import getgeneratorstate

getgeneratorstate(my_coro2)  # 未启动

'GEN_CREATED'

In [9]:
next(my_coro2)

-> Started: a = 14


14

In [10]:
getgeneratorstate(my_coro2)  # 暂停

'GEN_SUSPENDED'

In [11]:
my_coro2.send(28)

-> Received: b = 28


42

In [12]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [13]:
getgeneratorstate(my_coro2)  # 终止

'GEN_CLOSED'

无限循环表明：仅当调用方在协程上调用 .close() 方法，或者没有对协程的引用而被垃圾回收程序回收时，这个协程才会终止

In [14]:
# 计算移动平均值
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average  # 暂停执行协程
        total += term
        count += 1
        average = total/count

In [15]:
coro_avg = averager()  # 创建协程
next(coro_avg)  # 预激协程

coro_avg.send(10)  # 计算移动平均值

10.0

In [16]:
coro_avg.send(30)

20.0

In [17]:
coro_avg.send(5)

15.0

In [18]:
# 构建预激装饰器
from functools import wraps

def coroutine(func):
    """装饰器：向前执行到第一个 yield 表达式，预激 func"""
    @wraps(func)
    def primer(*args,**kwargs):  # 替换原来的生成器对象，返回预激的生成器对象
        gen = func(*args,**kwargs)  # 获取生成器对象
        next(gen)  # 预激
        return gen  # 返回
    return primer

In [19]:
@coroutine  # <5>
def averager():  # <6>
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [20]:
coro_avg = averager()  # 获取生成器对象
getgeneratorstate(coro_avg)  # 已经预激

'GEN_SUSPENDED'

In [21]:
coro_avg.send(10)

10.0

In [22]:
coro_avg.send(30)

20.0

In [23]:
coro_avg.send(5)

15.0

使用 yield from 句法调用协程时，会自动预激。标准库中的 asyncis.coroutine 装饰器不会预激协程。

协程中未处理的异常会向上冒泡，传递给 next 函数或 send 方法阿德调用方（即触发协程的对象）

In [24]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [25]:
coro_avg.send(50)

45.0

In [26]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

客户端代码可以在生成器对象上调用两个方法，显式地把异常发给协程
- generator.throw(exc_type\[, exc_value[, traceback)
    - 使生成器在暂停的 yield 表达式处抛出指定的异常
- generator.close()
    - 使生成器在暂停的 yield 表达式处抛出 GeneratorExit 异常

In [27]:
class DemoException(Exception):
    """DemoException 异常"""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:  # 特别处理
            print('*** DemoException handled. Continuing...')
        else:  # 没有异常
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')  # 这一行永远不会执行

激活和关闭 demo_exc_handling ，没有异常

In [28]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [29]:
exc_coro.send(11)

-> coroutine received: 11


In [30]:
exc_coro.send(22)

-> coroutine received: 22


In [31]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

把 DemoException 异常传入 demo_exc_handling 不会导致协程中止

In [32]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [33]:
exc_coro.send(11)

-> coroutine received: 11


In [34]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [35]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

如果无法处理传入的异常，协程会终止

In [36]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [37]:
exc_coro.send(11)

-> coroutine received: 11


In [38]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [39]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [40]:
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

激活和关闭 demo_finally ，没有异常

In [41]:
fin_coro = demo_finally()
next(fin_coro)

-> coroutine started


In [42]:
fin_coro.send(11)

-> coroutine received: 11


In [43]:
fin_coro.send(22)

-> coroutine received: 22


In [44]:
fin_coro.close()

-> coroutine ending


如果无法处理传入的异常，协程会终止，在终止时执行操作

In [45]:
fin_coro = demo_finally()
next(fin_coro)

-> coroutine started


In [46]:
fin_coro.send(11)

-> coroutine received: 11


In [47]:
fin_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: 

Python 引入 yield from 结构的主要原因之一与把异常传入嵌套的协程有关。另一个原因是让协程更方便地返回值。

某些协程不会产出值，而是在最后返回一个值（通常是某种累计值）。
- 为了返回值，协程必须正常终止

In [48]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break  # 终止
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # 返回值

In [49]:
coro_avg = averager()

# 预激
next(coro_avg)

# 不产出值
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

In [50]:
# 终止循环
# 生成器对象抛出 StopIteration 异常，异常对象的 value 属性保存着返回的值
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [51]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
    print(result)

Result(count=3, average=15.5)


对 yield from 结构来说，解释器不仅会捕获 StopIteration 异常，还会把 value 属性的值编程 yield from 表达式的值。

在生成器 gen 中使用 yield from subgen() 时，subgen 会获得控制权，把产出的值传给 gen 的调用方，即调用方可以直接控制 subgen ；与此同时，gen 会阻塞，等待 subgen 终止。

In [52]:
# yield
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [53]:
# yield from
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

In [54]:
# 使用 yield from 链接可迭代对象
def chain(*iterables):
    for it in iterables:
        yield from it

s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

yield from x 表达式对 x 对象所做的第一件事是，调用 iter(x) ，从中获取迭代器。

yield from 的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的样板代码。

PEP 380 使用了一些专门的术语
- 委派生成器
    - 包含 yield from <iterable> 表达式的生成器函数
    - 委派生成器也是迭代器
- 子生成器
    - 在 yield from 表达时中 <iterable> 部分获取的生成器
    - 子生成器也是迭代器
- 调用方
    - 指代调用委派生成器的客户端代码

![](https://github.com/fluentpython/images/blob/master/yield-channel.png?raw=true)

In [55]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield  # 绑定
        if term is None:  # 终止条件
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)  # 返回值


# 委派生成器
def grouper(results, key):
    while True:  # 每次迭代都新建一个 averager 实例；每个实例都作为协程使用的生成器对象
        results[key] = yield from averager()


# 调用方，客户端代码
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)  # group 作为协程使用
        next(group)  # 预激
        for value in values:
            group.send(value)  # 发送值
        group.send(None)  # 终止，执行下一个协程

#     print(results)  # uncomment to debug
    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

In [56]:
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


任何 yield from 链条都必须由客户驱动，在最外层委派生成器上调用 next(...) 函数或者 .send(...) 方法。可以隐式调用，例如使用 for 循环。

yield from 的行为
- 子生成器产出的值都直接传给委派生成器的调用方（即客户端代码）
- 使用 send() 方法发给委派生成器的值豆直接传给子生成器
- 生成器退出时，生成器（或子生成器）中的 return expr 表达式会触发 StopIteration(expr) 异常抛出
- yield from 表达式的值是子生成器终止时传给 StopIteration 异常的第一个参数

yield from 结构的另外两个特性与异常和终止有关
- 传入委派生成器的异常，除了 GeneratorExit 之外都传给子生成器的 throw() 方法
- 如果把 GeneratorExit 异常传入委派生成器，或者在委派生成器上调用 close() 方法，那么在子生成器上调用 close() 方法，如果它有的话

- [PEP 380 -- Syntax for Delegating to a Subgenerator](https://www.python.org/dev/peps/pep-0380/#formal-semantics)

简化的伪代码，等效于委派生成器中的 RESULT = yield from EXPR 语句，不支持 .throw() 和 .close() 方法，只处理 StopIteration 异常

```python
_i = iter(EXPR)  # 迭代器
try:
    _y = next(_i)  # 预激
except StopIteration as _e:
    _r = _e.value  # 返回值
else:
    while 1:
        _s = yield _y  # 子生成器产出
        try:
            _y = _i.send(_s)  # 转发
        except StopIteration as _e:  # 终止
            _r = _e.value
            break

RESULT = _r  # 最终结果
```

要预激子生成器，这表明用于自动预激的装饰器与 yield from 结构不兼容。

伪代码，等效于委派生成器中的 RESULT = yield from EXPR 语句

```python
_i = iter(EXPR)  # 迭代器，子生成器
try:
    _y = next(_i)  # 产出的值，子生成器产出的值
except StopIteration as _e:
    _r = _e.value  # 结果，最终的结果
else:
    while 1:  # <
        try:
            _s = yield _y  # 发送的值，调用方发给委派生层器的值
        except GeneratorExit as _e:  # 异常
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:  # 处理调用方通过 .throw(...) 方法传入的异常
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:  # <8>
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:  # 没有异常
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:  # 终止
                _r = _e.value
                break

RESULT = _r  # 最终结果
```

PEP 342：协程能自然地表述很多算法，例如仿真、游戏、异步 I/O，以及其他事件驱动型编程形式或协作式多任务。

协程是 asyncio 包的基础构建。

离散事件仿真（Discrete Event Simulation, DES）是一种把系统建模成一系列事件的仿真类型。

在离散事件仿真中，仿真“钟”向前推进的量不是固定的，而是直接推进到下一个事件模型的模拟时间。

连续仿真的仿真钟以固定的量（通常很小）不断向前推进。

回合制游戏就是离散事件仿真的例子：游戏的状态指在玩家操作时变化，而且一旦玩家决定下一步怎么走了，仿真钟就会冻结。

实施游戏则是连续仿真：仿真钟一直在运行，游戏的状态在一秒钟之内更新很多次，因此反应慢的玩家特别吃亏。

这两种仿真类型都能使用多线程或在单个线程中使用面向事件的编程技术（例如事件循环驱动的回调或协程）实现。为了实现连续仿真，在多个线程中处理实时并行的操作更自然。而协程恰好为实现离散事件仿真提供了合理的抽象。

在仿真领域，进程这个术语指代模型中某个实体的活动。

In [57]:
import random
import collections
import queue
import argparse


# 默认值
DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

# 事件，时间、编号、活动
Event = collections.namedtuple('Event', 'time proc action')


# 协程
def taxi_process(ident, trips, start_time=0):  # 每辆出租车调用一次，创建生成器对象，表示各辆出租车的运营过程
    """每次改变状态时创建事件，把控制权让给仿真器"""
    time = yield Event(start_time, ident, 'leave garage')  # 第一个事件是离开车库，协程暂停，等待
    for i in range(trips):  # 每次行程都会执行一遍
        time = yield Event(time, ident, 'pick up passenger')  # 产出一个事件，拉到乘客
        time = yield Event(time, ident, 'drop off passenger')  # 产出一个事件，乘客下车

    yield Event(time, ident, 'going home')  # 指定的行程数量完成后，产出事件，回家
    # 出租车进程结束 # 协程执行到最后，生成器对象抛出 StopIteration 异常


# 离散事件仿真类
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()  # 优先队列，按时间正向排序
        self.procs = dict(procs_map)  # 映射出租车编号

    def run(self, end_time):  # 参数：仿真结束时间
        """排定并显示事件，直到时间结束"""
        # 排定各个出租车的第一个事件
        for _, proc in sorted(self.procs.items()):  # 排序
            first_event = next(proc)  # 预激
            self.events.put(first_event)  # 添加事件

        # 仿真系统主循环
        sim_time = 0  # 仿真钟归零
        while sim_time < end_time:
            if self.events.empty():  # 事件全部完成
                print('*** end of events ***')
                break

            current_event = self.events.get()  # 当前事件（时间最早）
            sim_time, proc_id, previous_action = current_event  # 拆包
            print('taxi:', proc_id, proc_id * '   ', current_event)
            active_proc = self.procs[proc_id]  # 当前活动的出租车的协程
            next_time = sim_time + compute_duration(previous_action)  # 传入前一个动作，计算下一个事件的时间
            try:
                next_event = active_proc.send(next_time)  # 产出下一个事件
            except StopIteration:
                del self.procs[proc_id]  # 抛出异常，删除协程
            else:
                self.events.put(next_event)  # 否则，放入队列
        else:  # <16>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))


def compute_duration(previous_action):
    """使用指数分布计算活动的持续时间"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1


def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)


if __name__ == '__main__':

#     parser = argparse.ArgumentParser(
#                         description='Taxi fleet simulator.')
#     parser.add_argument('-e', '--end-time', type=int,
#                         default=DEFAULT_END_TIME,
#                         help='simulation end time; default = %s'
#                         % DEFAULT_END_TIME)
#     parser.add_argument('-t', '--taxis', type=int,
#                         default=DEFAULT_NUMBER_OF_TAXIS,
#                         help='number of taxis running; default = %s'
#                         % DEFAULT_NUMBER_OF_TAXIS)
#     parser.add_argument('-s', '--seed', type=int, default=None,
#                         help='random generator seed (for testing)')

#     args = parser.parse_args()
#     main(args.end_time, args.taxis, args.seed)
    
    main(DEFAULT_END_TIME, DEFAULT_NUMBER_OF_TAXIS, None)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 0  Event(time=5, proc=0, action='drop off passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 0  Event(time=6, proc=0, action='pick up passenger')
taxi: 1     Event(time=6, proc=1, action='pick up passenger')
taxi: 2        Event(time=10, proc=2, action='leave garage')
taxi: 2        Event(time=14, proc=2, action='pick up passenger')
taxi: 2        Event(time=16, proc=2, action='drop off passenger')
taxi: 1     Event(time=18, proc=1, action='drop off passenger')
taxi: 0  Event(time=24, proc=0, action='drop off passenger')
taxi: 1     Event(time=25, proc=1, action='pick up passenger')
taxi: 0  Event(time=29, proc=0, action='going home')
taxi: 1     Event(time=30, proc=1, action='drop off passenger')
taxi: 2        Event(time=33, proc=2, action='pick up passenger')
taxi: 1     Event(time=35, proc=1, action='pick up passenger')
taxi: 1     Event(tim

3 辆出租车的行程是交叉进行的，出租车每隔 5 分钟从车库出发。

离散事件仿真经常使用指数分布。

描述协程的操作时经常使用 drive 这个动词，例如：客户代码把值发给协程，驱动协程。

优先队列是离散事件仿真系统的基础构件：创建事件的顺序不定，放入这种队列之后，可以按照各个事件排定的时间顺序取出。

- 等效于调用 `main(end_time=120, seed=3)`

![](16-3.png)

In [58]:
# 表示一辆出租车
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)  # 预激

Event(time=0, proc=13, action='leave garage')

In [59]:
# 7min 后找到第一个乘客
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [60]:
# 23min 后第一个乘客下车
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [61]:
# 5min 后找到第二个乘客
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [62]:
# 48min 后第二个乘客下车
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [63]:
# 1 min 后回家
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [64]:
# 执行到协程末尾，返回后抛出异常
taxi.send(_.time + 10)

StopIteration: 

yield from subgenerator() 这个结构假定 subgenerator 没有预激，然后自动预激。

yield from 结构的三个主要组件
- 委派生成器（在定义体中使用 yield form）
- yield from 激活的子生成器
- 通过委派生成器中 yield form 表达式架设起来的通道把值发给子生成器，从而驱动整个过程的客户代码

事件驱动型框架（如 Tornado 和 asyncio）的运作方式：在单个线程中使用一个主循环驱动协程执行并发活动。

实现面向事件编程：
- 协作式多任务（协程）：协程显式自主地把控制权让步给中央调度程序
- 抢占式多任务（多线程）：调度程序可以在任何时刻暂停线程，把控制权让给其他线程

对编程语言来说，关键字的作用是建立控制流程和表达式计算的基本规则。